In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from sklearn.metrics import accuracy_score

In [2]:
BATCH_SIZE = 256
NUM_CLASSES = 10
EPOCHS = 50
NUM_MOD = 2
NUM_NETS = 30
VERBOSE = 2

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Guardamos la codificacion de clases explicitas inicial, nos hará falta
yy_test = y_test

# Convertimos los tensores de varias dimensiones a 1D (equivalente a Flatten)

x_train = np.reshape(x_train,(50000,3072))
x_test = np.reshape(x_test,(10000,3072))

# Normalizamos los valores de píxeles del rango 0-255 a 0.0-1.0

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Para problemas Multiclase conviene usar One Hot Encoding (y categorical_crossentropy+softmax)
# Cada valor de clase (0-9) se convierte en un vector de 0s y un 1

y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.layers import GaussianNoise, Dropout
from tensorflow.keras.regularizers import l2

In [5]:
# Creamos el conjunto de modelos a entrenar
for i in range(NUM_MOD):
    model = [0] * NUM_NETS
    for j in range(NUM_NETS):
            model[j] = Sequential()
            # model[j].add(GaussianNoise(0.1, input_shape=(3072,)))
            model[j].add(Dense(300, activation='relu', input_dim=3072))
            model[j].add(Dense(200, activation='relu'))
            # model[j].add(Dropout(0.5))
            model[j].add(Dense(NUM_CLASSES, activation='softmax'))
            model[j].summary()
            model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    NUM_NETS = NUM_NETS - 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               921900    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 984,110
Trainable params: 984,110
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 300)               921900    
_________________________________________________________________
dense_4 (Dense)              (None, 200)               60200     
____________________________

In [6]:
# Este es el modelo de prueba con 1 epoch para comprobar que todo va bien
NUM_NETS = 30
for i in range(NUM_MOD):
    history = [0] * NUM_NETS
    for j in range(NUM_NETS):
            print('NET ', j+1)
            history[j] = model[j].fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = 1, verbose = 1)
            score = model[j].evaluate(x_test, y_test, verbose = VERBOSE)
            print('Test accuracy:', score[1])
    NUM_NETS = NUM_NETS - 1

NET  1
196/196 [==============================] - 1s 3ms/step - loss: 1.9290 - accuracy: 0.3107

IndexError: list index out of range

In [7]:
# Este es el modelo completo con todas las epochs
history = [0] * NUM_NETS

for j in range(NUM_NETS):
        print('NET ', j+1)
        history[j] = model[j].fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = 0)
        score = model[j].evaluate(x_test, y_test, verbose = VERBOSE)
        print('Test accuracy:', score[1])

NET  1
313/313 - 0s - loss: 1.4821 - accuracy: 0.4931
Test accuracy: 0.49309998750686646
NET  2
313/313 - 0s - loss: 1.4882 - accuracy: 0.4866
Test accuracy: 0.48660001158714294
NET  3
313/313 - 0s - loss: 1.5090 - accuracy: 0.4772
Test accuracy: 0.4772000014781952
NET  4
313/313 - 0s - loss: 1.4698 - accuracy: 0.4916
Test accuracy: 0.49160000681877136
NET  5
313/313 - 0s - loss: 1.4987 - accuracy: 0.4840
Test accuracy: 0.48399999737739563
NET  6
313/313 - 0s - loss: 1.4714 - accuracy: 0.4926
Test accuracy: 0.4925999939441681
NET  7
313/313 - 0s - loss: 1.4893 - accuracy: 0.4846
Test accuracy: 0.4846000075340271
NET  8
313/313 - 0s - loss: 1.4885 - accuracy: 0.4854
Test accuracy: 0.48539999127388
NET  9
313/313 - 0s - loss: 1.4814 - accuracy: 0.4930
Test accuracy: 0.49300000071525574
NET  10
313/313 - 0s - loss: 1.4900 - accuracy: 0.4836
Test accuracy: 0.483599990606308
NET  11
313/313 - 0s - loss: 1.4739 - accuracy: 0.4909
Test accuracy: 0.4909000098705292
NET  12
313/313 - 0s - loss:

In [8]:
resultados = np.zeros((y_test.shape[0], NUM_CLASSES))

for j in range(NUM_NETS):
        y_pred = model[j].predict(x_test)
        resultados = resultados + y_pred
# Reconvertimos la codificacion One Shot a una etiqueta de clase (0-9)
        y_pred = np.argmax (y_pred, axis=1)
        print ('Accuracy NET ', j+1, accuracy_score(yy_test, y_pred))

Accuracy NET  0 0.4931
Accuracy NET  1 0.4866
Accuracy NET  2 0.4772
Accuracy NET  3 0.4916
Accuracy NET  4 0.484
Accuracy NET  5 0.4926
Accuracy NET  6 0.4846
Accuracy NET  7 0.4854
Accuracy NET  8 0.493
Accuracy NET  9 0.4836
Accuracy NET  10 0.4909
Accuracy NET  11 0.505
Accuracy NET  12 0.4884
Accuracy NET  13 0.4882
Accuracy NET  14 0.496
Accuracy NET  15 0.4814
Accuracy NET  16 0.4793
Accuracy NET  17 0.4881
Accuracy NET  18 0.4893
Accuracy NET  19 0.4903
Accuracy NET  20 0.4865
Accuracy NET  21 0.4937
Accuracy NET  22 0.4942
Accuracy NET  23 0.488
Accuracy NET  24 0.4965
Accuracy NET  25 0.4995
Accuracy NET  26 0.4926
Accuracy NET  27 0.4879
Accuracy NET  28 0.4932
Accuracy NET  29 0.4918


In [9]:
# La suma de salidas la reconvertimos de One Shot Encoding a la etiqueta de clase correspondiente (0-9)
resultados = np.argmax(resultados, axis=1)
print ('Accuracy Ensemble ', accuracy_score(yy_test, resultados))


Accuracy Ensemble  0.5384
